#### Generate heat map of all data from 2028-2024

Select "run all" to run cells in order and view the full map below.

In [4]:
import pandas as pd
import geopandas as gpd
from sodapy import Socrata
import folium
from folium import plugins

In [5]:
##query Open Data API 
client = Socrata("data.cityofnewyork.us", None)
results = client.get("wz6d-d3jb", limit=10000) 

#turn results into a dataframe
results_df = pd.DataFrame.from_records(results) 

In [6]:
#combine each individual dataframe row's latitude and longitude columns into a single geometry point
#this makes each entry a plottable point on our map.
geo_df = gpd.GeoDataFrame(
    results_df, geometry=gpd.points_from_xy(results_df["longitude"], results_df["latitude"], crs=4326)
)

#loop through each geometry point and coalesce them together to make our heatmap
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

#determine center of base map (NYC's latitude and longitude)
map_center = [40.7128, -74.0060]

#declare a map variable for our base map of NYC
m = folium.Map(location=map_center, zoom_start=13)

#overlay heatmap onto our base map
plugins.HeatMap(heat_data).add_to(m)

#return map and look at it!
m